# Área de imports

# Carregamento dos dados

# Modelagem

Observação importante: 
- A utilização de uma GPU com CUDA habilitado é fortemente indicada para esta parte do tutorial e para a realização do projeto
- A ferramenta online [Google Colaboratory](https://colab.research.google.com/) disponibiliza gratuitamente um ambiente com GPU + CUDA para utilização em notebooks
- Há um tutorial detalhado de como utilizar o Google Colab no repositório deste tutorial